In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
from tensorflow import keras


In [2]:
categories={}
categories={0: 'अ', 1: 'आ', 2: 'इ', 3: 'ई', 4: 'उ', 5: 'ऊ', 6: 'ए', 7: 'ऐ', 8: 'ओ', 9: 'औ', 10: 'अं', 11: 'अः', 12: 'क', 13: 'ख', 14: 'ग', 15: 'घ', 16: 'ङ', 17: 'च', 18: 'छ', 19: 'ज', 20: 'झ', 21: 'ञ', 22: 'ट', 23: 'ठ', 24: 'ड', 25: 'ढ', 26: 'ण', 27: 'त', 28: 'थ', 29: 'द', 30: 'ध', 31: 'न', 32: 'प', 33: 'फ', 34: 'ब', 35: 'भ', 36: 'म', 37: 'य', 38: 'र', 39: 'ल', 40: 'व', 41: 'श', 42: 'ष', 43: 'स', 44: 'ह', 45: 'क्ष', 46: 'त्र', 47: 'ज्ञ', 48: 'ा'}

In [3]:
def line_detect(img):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray= cv2.GaussianBlur(gray, (5, 5), 0)
    gray=cv2.medianBlur(gray,5)
    rect,thresh=cv2.threshold(gray,125,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    minLineLength = 100
    maxLineGap = 10
    lines = cv2.HoughLinesP(thresh,1,np.pi/180,100,minLineLength,maxLineGap)
    if(len(lines[0][0])!=0):
        x1,y1,x2,y2=lines[0][0]
        return x1,y1,x2,y2
    else:
        return 0,0,0,0

In [4]:
def adjust(x1,y1,x2,y2,img):
    mid_width=img.shape[1]//2
    mid_height=img.shape[0]//2
    if(y1<mid_height and y2<mid_height):
        img=img
    elif(x1<mid_width and x2<mid_width):
        img=cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    elif(x1>mid_width and x2>mid_width):
        img=cv2.rotate(img,cv2.ROTATE_90_COUNTERCLOCKWISE)

    elif(y1>mid_height and y2>mid_height):
        img=cv2.flip(img,1)
        img=cv2.flip(img,0)
    return img
        
    
    


In [5]:
def crop_image(x1,y1,x2,y2,w1,z1,w2,z2,img):
    return img[y1+3:max(z1,z2),x1:max(x2,z2)]


In [6]:
def image_segmentation(crop):
    height=crop.shape[0]
    width=crop.shape[1]
    crop1=cv2.cvtColor(crop,cv2.COLOR_RGB2GRAY)
    rect,thresh=cv2.threshold(crop1,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    
    def value(item):
        return item[0]
    contours, hier = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    coordinate_set=[]
    character_set=[]
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        if(height-100<h<=height+20 and width//4<w<=width):
            cv2.rectangle(thresh, (x, y), (x+w, y+h), (254, 255, 254), 8)
            coordinate_set.append((x,x+w,y,y+h))
    coordinate_set=sorted(coordinate_set,key=value)
    for x in coordinate_set:
        character_set.append(crop[x[2]:x[3],x[0]:x[1]].copy())
    return character_set

In [ ]:
if __name__=="__main__":
    string=""
    original_height=100
    original_weight=100
    model = keras.models.load_model("tessaract.h5")
    path=str(input("please enter img path:"))
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    x1,y1,x2,y2=line_detect(img)
    adjusted_image=adjust(x1,y1,x2,y2,img)
    crop_x1,crop_y1,crop_x2,crop_y2=line_detect(adjusted_image)
    crop_w1,crop_z1,crop_w2,crop_z2=line_detect(cv2.flip(adjusted_image,0))
    crop=crop_image(crop_x1,crop_y1,crop_x2,crop_y2,crop_w1,crop_z1,crop_w2,crop_z2,adjusted_image)
    character_set=image_segmentation(crop)
    for x in character_set:
        x=cv2.cvtColor(x,cv2.COLOR_BGR2GRAY)
        rect,x=cv2.threshold(x,125,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        x=cv2.resize(x,(32,32))
        x= cv2.morphologyEx(x, cv2.MORPH_CLOSE,None,iterations=1)
        new_img=x.reshape(-1,32,32,1)
        new_img=new_img/255
        value=categories[np.argmax(model.predict(new_img))]
        string=string+str(value)
        print(value)
        plt.imshow(x)
        plt.show()
    print(string)